# Data sources

This section describes the tools implemented in Spark for saving, loading, and versioning the data.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark_session = (
    SparkSession.builder.appName("Temp")
    .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse")
    .getOrCreate()
)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/22 10:39:39 WARN Utils: Your hostname, user-ThinkPad-E16-Gen-2, resolves to a loopback address: 127.0.1.1; using 10.202.22.210 instead (on interface enp0s31f6)
25/09/22 10:39:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/22 10:39:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/09/22 10:39:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Saving

Spark has a set of methods for saving data frames for future use. Consider the most important ones. For that data frame has an attribute `write` that refers to a set of methods: `parquiet`, `csv`, and `json`; with really transparent naming.

There is also a `saveAsTable` method, that saves the data to the special storage managed by the Spark. This storage could be: `Hive Metastore` or `DeltaLake`.

---

Consider saving the simple table as a JSON table.

In [2]:
df = spark_session.createDataFrame(
    data=[("Alice", 25), ("Bob", 30), ("Cathy", 35)],
    schema=["Name", "Age"]
)

data_path = "/tmp/my_data"
df.write.json(data_path, mode='overwrite')

A set of `.json` files is created in the destination folder to store the data.

In [3]:
import os
os.listdir(data_path)

['part-00011-87a1ac33-4c79-4760-b7f5-16147b167620-c000.json',
 '.part-00000-87a1ac33-4c79-4760-b7f5-16147b167620-c000.json.crc',
 '.part-00005-87a1ac33-4c79-4760-b7f5-16147b167620-c000.json.crc',
 '.part-00011-87a1ac33-4c79-4760-b7f5-16147b167620-c000.json.crc',
 '._SUCCESS.crc',
 'part-00000-87a1ac33-4c79-4760-b7f5-16147b167620-c000.json',
 'part-00017-87a1ac33-4c79-4760-b7f5-16147b167620-c000.json',
 '.part-00017-87a1ac33-4c79-4760-b7f5-16147b167620-c000.json.crc',
 'part-00005-87a1ac33-4c79-4760-b7f5-16147b167620-c000.json',
 '_SUCCESS']

Consider important for now:

In [4]:
import json

for file_name in os.listdir(data_path):
    if file_name.endswith(".json"):
        with open(data_path + "/" + file_name, "r") as f:
            try:
                print(json.load(f))
            except: pass

{'Name': 'Bob', 'Age': 30}
{'Name': 'Cathy', 'Age': 35}
{'Name': 'Alice', 'Age': 25}


## Read csv

Use the `read.csv` method of the spark session to read a CSV file.

---

The following cell reads the `spark.csv` file that I prepared earlier.

In [5]:
df = spark_session.read.csv(
    "data_sources_files/scv_example.csv",
    header=True,
    inferSchema=True,
    multiLine=True,
    escape=','
)
display(df)

DataFrame[Name: string,  Age: double,  Salary: double]

### Shcema

Use the `schema` argument to define the schema. The schema can be specified as a simple string that matches column names with their expected data types.

---

The following cell shows the matching of the `int` data type to the `Age` column instead of the default `double` data type.

In [6]:
schema = """
Name string,
Age int,
Salary double
"""

spark_session.read.csv(
    "data_sources_files/scv_example.csv",
    schema=schema
)

DataFrame[Name: string, Age: int, Salary: double]

## SQL catalog

The *Spark SQL catalog* is a special file system that provies SQL access and data is described by a special metadata provided by PySpark. This section demonstrates how to access the capabilitites of the SQL catalog from the python SDK.

---

The SQL catalog is specified by the `spark.sql.warehouse.dir` attribute in the Spark configuration. The following cell displays the SQL catalog for the current Spark session.

**Note** It should be specified when creating of the session.

In [2]:
spark_session.conf.get("spark.sql.warehouse.dir")

'file:/tmp/spark-warehouse'

The following cell uses the `write.saveAsTable` method to store the data frame.

In [ ]:
df = spark_session.createDataFrame(
    data=[("Alice", 25), ("Bob", 30), ("Cathy", 35)],
    schema=["Name", "Age"]
)
df.write.saveAsTable("example_save")

The corresponding folder should now be in the warehouse storage:

In [5]:
!ls /tmp/spark-warehouse

example_save


This folder contains the partitions of the saved dataset.

In [6]:
!ls /tmp/spark-warehouse/example_save

part-00000-74ba1525-2161-4c89-94b1-c925b52a41ff-c000.snappy.parquet
part-00005-74ba1525-2161-4c89-94b1-c925b52a41ff-c000.snappy.parquet
part-00011-74ba1525-2161-4c89-94b1-c925b52a41ff-c000.snappy.parquet
part-00017-74ba1525-2161-4c89-94b1-c925b52a41ff-c000.snappy.parquet
_SUCCESS
